___
**Note: Code has been modified and repurposed to work locally and with local hardware and computer.**  
**Original code obtained from [Hybrid Demucs from Colab](https://colab.research.google.com/drive/1dC9nVxk3V_VPjUADsnFu8EiT-xnU1tGH?usp=sharing#scrollTo=79JbZGcAqX3p)**  
**Reference Model Source from GitHub [Demucs Music Source Separation](https://github.com/facebookresearch/demucs)**
___

# Import Required Libraries
___

In [1]:
# Torch for deep learning models
import torch

# File and directory handling
from pathlib import Path
from shutil import rmtree

# Subprocess for running shell commands
import subprocess as sp

# System-specific parameters and functions
import sys

# Check if CUDA (GPU) is available, otherwise fallback to CPU
device = torch.device(f"cuda:{torch.cuda.current_device()}" if torch.cuda.is_available() else "cpu")

#### Model and File Type Options

- `model`:
    - `"htdemucs` is a pre-trained model for separating different sources (like vocals, drums, other, bass) from a mixture.   
    - You can change this to another model if you want to experiment with different separation models, such as "demucs" or any other available model.  
    - List of models can be found here: [MODEL LIST HERE](https://github.com/facebookresearch/demucs/tree/main/demucs/remote)

- `extensions`: 
    - This is a list of audio file types the script will recognize and process.
    - You can add or remove file types based on your needs.

- `two_stems`: Stem(s) to seperate
    - You can set this to a specific stem like "vocals", "drums", or any other stem if you want to separate just one.
    - If left as None, it will separate all stems (e.g., vocals, drums, bass, etc.)

In [2]:
# Define the model to be used for separation
model = "htdemucs_ft"

# Define which audio file extensions to look for
extensions = ["mp3", "wav", "ogg", "flac"]  

# Define which stem(s) to separatee
two_stems = None 

#### Output Audio and File Paths Options

- `mp3`: 
  - This flag controls whether the output is in mp3 format or not.
  - If True, the output will be in mp3 format (as specified below).
  - If False, the output will be in a different format (e.g., wav).

- `mp3_rate`: 
  - This defines the bitrate for the mp3 output.
  - It's measured in kilobits per second (kbps).
  - Higher values generally result in better audio quality but larger file sizes.
  - 320 kbps is considered high-quality for mp3 audio.

- `float32`: 
  - This flag controls whether the output should be in 32-bit float format.
  - This is generally used for high-quality audio but results in larger file sizes.
  - If mp3 is True, this option is ignored.

- `int24`: 
  - Similar to float32, this flag controls whether the output is in 24-bit integer format.
  - It provides high-quality audio but is also larger in size compared to lower bit depths (e.g., 16-bit).

- `in_path`: 
  - This is the path where your input audio files are stored.
  - You can change the path to any directory on your system that contains the files you want to process.

- `out_path`: 
  - This is the path where the separated audio files will be saved after processing.
  - You can change this to any directory on your system where you want to store the output.

In [3]:
# Define options for the output audio
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.

# Set your input and output paths for local files
in_path = Path('./input_audio/')  # replace with your input folder
out_path = Path('./output_audio/')  # replace with your output folder


# Audio Handling
___

#### Finding Audio Files in the Input Directory

- `find_files(input_directory)`:
  - This function searches for audio files in the specified input directory.

  - **Parameters**:
    - `input_directory`: The directory where the function will look for audio files.

  - **Process**:
    - Iterates over all files in the directory specified by `input_directory`.
    - Checks if the file extension is in the list of extensions we want to process.
    - Adds the file to the list if it matches the desired extensions.
    
  - **Returns**:
    - A list of audio files found in the input directory.

In [4]:
from pathlib import Path

# Define the find_files function to find audio files in the input directory
def find_files(input_directory):
    audio_files = []

    # Path(in_path).iterdir() iterates over all files in the directory specified by in_path
    for file in Path(input_directory).iterdir():

        # Check if the file extension is in the list of extensions we want to process
        if file.suffix.lower().lstrip(".") in extensions:
            audio_files.append(file)
    return audio_files

#### Handling Subprocess Output Streams

- `copy_process_streams(process)`:
  - This function handles the output and error streams of a subprocess.

  - **Parameters**:
    - `process`: The subprocess whose streams are to be handled.

  - **Process**:
    - Uses `communicate()` to capture `stdout` (standard output) and `stderr` (standard error) of the process.
    - Prints the captured outputs to the console.
    
  - **Output**:
    - Writes the standard output and error messages to the console.

In [5]:
import sys

# Define the copy_process_streams function
def copy_process_streams(process: sp.Popen):
    # communicate() handles the I/O streams of the process, capturing stdout and stderr
    #  stdout means standard output, which is the console output of the process
    #  stderr means standard error, which is the error output of the process
    stdout, stderr = process.communicate()

    # Print the outputs to the console
    if stdout:
        # Write standard output (stdout) to the console
        sys.stdout.write(stdout.decode())
    if stderr:
        # Write errors (stderr) to the console
        sys.stderr.write(stderr.decode())

#### Main Separation Function

- `separate(inp=None, outp=None)`:
  - This function separates audio files using the Demucs model.

  - **Parameters**:
    - `inp`: The input directory containing audio files (default is `in_path`).
    - `outp`: The output directory where separated files will be saved (default is `out_path`).

  - **Process**:
    - Constructs the command to execute the Demucs model for separation.
    - Adds options to the command based on specified conditions.
    - Finds all audio files in the input directory.
    - Prints the list of files to be processed and the full command being executed.
    - Executes the separation command using `subprocess`.
    - Handles the process output (stdout, stderr) and waits for the process to finish.
    - Checks the return code to see if the command ran successfully.

In [6]:
import subprocess as sp

# The main separation function to separate audio files
def separate(inp=None, outp=None):
    inp = inp or in_path  # Use the input path or default to 'in_path'
    outp = outp or out_path  # Use the output path or default to 'out_path'

    # Create the command that will execute the Demucs model for separation
    cmd = ["python", "-m", "demucs.separate", "-o", str(outp), "-n", model, "--device", f"{device}"]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    
    # Find all the audio files in the input directory
    files = [str(f) for f in find_files(inp)]

    if not files:
        print(f"[ERROR] No valid audio files in {in_path}")
        return
    
    # Print the list of files that will be processed
    print("[INFO] \n\tGoing to separate the files:")
    for idx, file in enumerate(files, start=1):
        print(f"\t\t{idx}. {file}")
    print("═" * 90)                       
    
    # Print the full command being executed
    print(f"[CMD] \n\t`{' '.join(cmd)}`")
    print("═" * 90)

    # Execute the separation command using subprocess
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)

    # Handle the process output (stdout, stderr)
    copy_process_streams(p)

    # Wait for the process to finish
    p.wait()

    # Check the return code to see if the command ran successfully
    if p.returncode != 0:
        print(f"[ERROR] Command failed, something went wrong.")

#### Handling File Uploads

- `from_upload()`:
  - This function handles file uploads for local systems or web interfaces.

  - **Process**:
    - Sets the input and output directories.
    - Removes and recreates the input and output directories if they already exist.
    - Calls the `separate` function to process the audio files.

In [7]:
"""
from pathlib import Path

# Function to handle file upload for local system or web interface
def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')
    
    if in_path.exists():
        # Remove the input directory if it already exists
        rmtree(in_path)
    # Create the input directory
    in_path.mkdir()
    
    if out_path.exists():
        # Remove the output directory if it already exists
        rmtree(out_path)
    # Create the output directory
    out_path.mkdir()
    
    # # Your local files will already be present, so we don't need to upload files.
    separate(in_path, out_path)
"""

"\nfrom pathlib import Path\n\n# Function to handle file upload for local system or web interface\ndef from_upload():\n    out_path = Path('separated')\n    in_path = Path('tmp_in')\n    \n    if in_path.exists():\n        # Remove the input directory if it already exists\n        rmtree(in_path)\n    # Create the input directory\n    in_path.mkdir()\n    \n    if out_path.exists():\n        # Remove the output directory if it already exists\n        rmtree(out_path)\n    # Create the output directory\n    out_path.mkdir()\n    \n    # # Your local files will already be present, so we don't need to upload files.\n    separate(in_path, out_path)\n"

# Audio Processing
___

#### Calling the Separation Function

- `separate()`:
  - This call initiates the separation process for all audio files in the input directory.
  
  - **Output**:
    - Prints the progress and results of the separation process, including the list of files being processed and the command being executed.

In [8]:
# Call the separate function to process all audio files in your input directory
separate()

[INFO] 
	Going to separate the files:
		1. input_audio\input_4.mp3
══════════════════════════════════════════════════════════════════════════════════════════
[CMD] 
	`python -m demucs.separate -o output_audio -n htdemucs_ft --device cuda:0 --mp3 --mp3-bitrate=320`
══════════════════════════════════════════════════════════════════════════════════════════
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in C:\Users\chris\Code\AI-UTSA-2024\Projects\Group_3_Project\notebooks\christian_exploration\output_audio\htdemucs_ft
Separating track input_audio\input_4.mp3


100%|██████████████████████████████████████████████████████████████████████| 274.95/274.95 [00:09<00:00, 28.54seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 274.95/274.95 [00:07<00:00, 36.33seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 274.95/274.95 [00:07<00:00, 36.06seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 274.95/274.95 [00:07<00:00, 34.89seconds/s]
